<a href="https://colab.research.google.com/github/luisgdelafuente/gnai/blob/main/embeddings_chromadb%2BPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken

In [ ]:
import requests

text_url = 'https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt'
response = requests.get(text_url)

#let'extract only the text from the response
data = response.text

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(data)

In [ ]:
len(texts)
texts[0],texts[1]

('Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.',
 'Groups of citizens blocking tanks with

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
persist_directory = 'db'
docsearch = Chroma.from_texts(
    texts, 
    embeddings,
    persist_directory = persist_directory,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))]
    )

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
from langchain import OpenAI

#convert the vectorstore to a retriever
retriever=docsearch.as_retriever()

In [ ]:
retriever.search_type

'similarity'

In [ ]:
docs = retriever.get_relevant_documents("What did the president say about Russia")
len(docs)
docs

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.', metadata={'source'

In [ ]:
#create the chain to answer questions
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm = OpenAI(temperature=0), 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents = True
    )

In [ ]:
def process_result(result):
  print(result['answer'])
  print("\n\n Sources : ",result['sources'] )
  print(result['sources'])

In [ ]:
question = "What did the president say about Russia? please answer in 1-2 sentences"
result = chain({"question": question})
process_result(result)

 The president said that Russia's Vladimir Putin had sought to shake the foundations of the free world, and that the US and its allies would defend every inch of NATO countries with the full force of their collective power, and that Putin would pay a continuing high price over the long run.



 Sources :  0-pl, 4-pl, 5-pl
0-pl, 4-pl, 5-pl
